# Рекомендация тарифов

## Описание проекта


Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных уже сделана.

Необходимо построить модель с максимально большим значением accuracy. Нужно довести долю правильных ответов по крайней мере до 0.75 и проверить accuracy на тестовой выборке.

**Путь к файлу:**
- /datasets/users_behavior.csv

**Описание данных таблицы:**
- Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**План действий:**

1. Ознакомление с данными.
2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.
3. Исследование качество разных моделей, меняя гиперпараметры.
4. Проверка качество модели на тестовой выборке.
5. Дополнительно: проверка модели на вменяемость.

### Глава 1. Открытие файла с данными и изучение общей информации

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# чтение файла с данными 
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('users_behavior.csv')

In [3]:
# просмотр и изучение данных
df.info()
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


5 столбцов, 3214 строк, нулевых строк нет

### Глава 2. Разделение исходные данные на обучающую, валидационную и тестовую выборки.

In [4]:
features = df.drop('is_ultra', axis=1) # извлекаем признаки
target = df['is_ultra'] # извлекаем целевой признак

 
# Разделение исходных данных на обучающую и валидационную и тестовую выборки
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, stratify=target, test_size=0.25, random_state=12345)

# Далее разделяем обучающую и валидационную выборку на отдельные обучающую (features_train, target_train) и валидационную (features_valid, target_valid) выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, target_train_valid, stratify=target_train_valid, test_size=0.25, random_state=12345)


In [20]:
display(features_test.shape)
target_test.value_counts()

(804, 4)

0    558
1    246
Name: is_ultra, dtype: int64

In [21]:
display(features_train.shape)
target_train.value_counts()

(1807, 4)

0    1253
1     554
Name: is_ultra, dtype: int64

In [22]:
display(features_valid.shape)
target_valid.value_counts()

(603, 4)

0    418
1    185
Name: is_ultra, dtype: int64

In [23]:
#!pip install fast_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 668.6 kB/s eta 0:00:00 0:00:01


In [24]:
#from fast_ml.model_development import train_valid_test_split
#X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target='is_ultra',
                                                                           train_size=0.6, valid_size=0.2, test_size=0,2)

### Глава 3. Исследование качества разных моделей

*Данная задача относиться к задачам классификации, в нашем случае бинарной классификации.*

Рассмотрим 2 модели:

- Логистическая регрессия LogisticRegression из библиотеки sklearn.linear_model
- Случайный лес RandomForestClassifier из библиотеки sklearn.ensemble

Сразу введем псевдослучайность для алгоритма обучения, используя генератор псевдослучайных чисел random_state. Таким образом модель будет воспринимать данные как случайные, для нас это необходимо , чтобы результаты неизменно получались одинаковыми (другими словами, повторить удачный эксперимент).

### LogisticRegression

In [5]:
from sklearn.linear_model import LogisticRegression
from joblib import dump


model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=300)
model_lr.fit(features_train, target_train) #скармливаем параметры

# Оценка качества модели на обучающей выборке
train_accuracy_lr = model_lr.score(features_train, target_train)
print('Accuracy тренирочоной модели:', train_accuracy_lr) #выводим accuracy тренировочной модели 

# Оценка качества модели на валидационной выборке
valid_accuracy_lr = model_lr.score(features_valid, target_valid)
print('Accuracy валидационной модели:',valid_accuracy_lr) #выводим accuracy валидационной модели

# Сохраним модель для дальнейшего использования
dump(model_lr, 'model_log_reg.joblib')


Accuracy тренирочоной модели: 0.7487548422800221
Accuracy валидационной модели: 0.7412935323383084


['model_log_reg.joblib']

### Случайный лес

In [6]:
from sklearn.ensemble import RandomForestClassifier

best_model_rf = None
best_result_accuracy_rf = 0

# Создание и обучение модели случайного леса с алгоритмом поиска лучшего количества оценщиков
for est in range(1, 10):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=est) # обучим модель с заданным количеством деревьев
    model_rf.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result_accuracy_rf = model_rf.score(features_valid, target_valid) # посчитайте качество модели на валидационной выборке
    if result_accuracy_rf > best_result_accuracy_rf:
        best_model_rf = model_rf # сохраните наилучшую модель
        best_result_accuracy_rf = result_accuracy_rf #  сохраните наилучшее значение метрики accuracy на валидационных данных
        
        

print("Accuracy наилучшей модели на валидационной выборке:", best_result_accuracy_rf)

# Сохраним модель для дальнейшего использования
dump(best_model_rf, 'model_random_forest.joblib')

Accuracy наилучшей модели на валидационной выборке: 0.7993366500829188


['model_random_forest.joblib']

## Проверим модель на тестовой выборке

In [8]:
# Оценка качества модели логистической регресс на тестовой выборке
test_accuracy_lr = model_lr.score(features_test, target_test)
print('Accuracy тестовой модели (логистической регрессии):', test_accuracy_lr)

# Оценка качества модели случайного леса на тестовой выборке
test_accuracy_rf = model_rf.score(features_test, target_test)
print('Accuracy тестовой модели (случайный лес):', test_accuracy_rf)


Accuracy тестовой модели (логистической регрессии): 0.7611940298507462
Accuracy тестовой модели (случайный лес): 0.7723880597014925


## Вывод

**Модель Случайный лес показала более хорошие результаты из 2 моделей.**
- Полученные значения качества модели в 77% говорит о хорошем предсказании.
- Построенная модель для задачи классификации, сможет подобрать для пользователей, пользующихся архивными тарифами, более подходящий тариф («Смарт» или «Ультра»).
- Проанализировав поведение клиентов: тратах минут, сообщений и интернет трафика, модель в 4 из 5 случаев предскажет верный тариф для пользователя.
- Модель подходит как система "Рекомендации тарифов"